## Retrieval from Vector Database and Basic Question Answering  
Starts the same as basic_retrieval.ipynb

In [9]:
!source ../.rag_venv/bin/activate

In [10]:
# Imports 
import sys
sys.path.append('../')

from VectorDatabase import VectorDatabase
from Document       import Document
from os             import listdir
from os.path        import isfile, join

In [11]:
# Initialise vector database
v_db = VectorDatabase()
v_db.setClient(persistent=False)
v_db.createCollection("Bible_John")

In [12]:
# Add documents into vector database
dataset_folder = "documents"

def check_is_txt(file):
    if (isfile(join(dataset_folder, file))) and (file.endswith('.txt')):
        return True

metadatas = [{"filepath": join(dataset_folder,f), 
              "book"    :(f.split('_')[0]), 
              "chapter" :(f.split('_')[1].split('.')[0])
              } for f in listdir(dataset_folder) if check_is_txt(f)]

In [13]:
metadatas

[{'filepath': 'documents/john_3.txt', 'book': 'john', 'chapter': '3'},
 {'filepath': 'documents/john_2.txt', 'book': 'john', 'chapter': '2'},
 {'filepath': 'documents/john_1.txt', 'book': 'john', 'chapter': '1'},
 {'filepath': 'documents/john_5.txt', 'book': 'john', 'chapter': '5'},
 {'filepath': 'documents/john_4.txt', 'book': 'john', 'chapter': '4'}]

In [14]:
# initialise lists to be added to collection
doc_list     = []
doc_metadata = []
embeddings   = []

In [15]:
# read in each dataset file
for item in metadatas:
    path = item["filepath"]

    doc  = Document(item["filepath"])

    doc_list.append(doc.fulltext)
    doc_metadata.append(item)
    embeddings.append(doc.embedding[0])

In [16]:
print(len(doc_list) == len(doc_metadata))

True


In [17]:
# add to collection
v_db.addToCollection(docs   = doc_list, 
                     embeds = embeddings,
                     meta   = doc_metadata, 
                     ids    = [f"id{i}" for i in range(len(doc_list))])

### Ollama  
#### Download Model with Ollama
Install Ollama: https://ollama.com  
Set directory to save models in: https://dev.to/hamed0406/how-to-change-place-of-saving-models-on-ollama-4ko8  
Download a model (be wary of model size)  

I will use Deepseek-R1:1.5b (1.1GB) found here: https://ollama.com/library/deepseek-r1:1.5b

#### Serve Ollama
optional:   
export OLLAMA_HOST=127.0.0.1 # environment variable to set ollama host  
export OLLAMA_PORT=11434     # environment variable to set the ollama port  

start serving ollama:  
ollama serve  

In [18]:
model_name = 'deepseek-r1:1.5b'

In [19]:
from langchain.schema            import BaseRetriever
from langchain.docstore.document import Document
from langchain.prompts           import PromptTemplate
from langchain_community.llms    import Ollama
from langchain.chains            import RetrievalQA

In [20]:
# allow custom retrieval function
class CustomRetriever(BaseRetriever):
    def __init__(self, retrieval_function, num_docs_to_retrieve):
        super().__init__()
        self._retrieval_function   = retrieval_function
        self._num_docs_to_retrieve = num_docs_to_retrieve
        
    def _get_relevant_documents(self, query: str) -> list:
        retrieved      = self._retrieval_function(query, self._num_docs_to_retrieve)
        num_of_results = len(retrieved['ids'][0])
        results        = []
        for i in range(num_of_results):
            results.append(Document(
                                page_content = retrieved['documents'][0][i],
                                metadata     = retrieved['metadatas'][0][i]
                                ))
        return results

# main pipeline
def RAG_pipeline(model_name, retriever_func, num_docs):
    # Custom prompt template
    prompt_template = """
    Use only the following pieces of context to answer the question at the end. 
    If you don't know the answer based only on the context, just say you don't know.

    Context:
    {context}

    Question: {question}
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    retriever = CustomRetriever(retriever_func, num_docs)
    llm       = Ollama(model=model_name)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return qa_chain

In [21]:
qa_chain = RAG_pipeline(model_name, v_db.queryDatabase, 2)

/var/folders/qd/x9tnwk2519v14dprnysc0tpw0000gn/T/ipykernel_13407/4283359118.py:38: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm       = Ollama(model=model_name)


In [71]:
response = qa_chain.invoke("Who is Jesus?") # response takes ~1 minute

In [72]:
print("***** Model Output *****")
print(response['result'])

print("\n***** Source(s) *****")
for source in response['source_documents']:
    print(source.metadata)

***** Model Output *****
<think>
Okay, so I'm trying to figure out who Jesus is. From what I've heard, there are different people mentioned in this text, but the question is asking specifically about Jesus. Let me go through each part and see how that makes sense.

First, there's a man named Jonathan who tells Jesus to be the Messiah. That seems like an important clue because the Messiah was Jesus. Then, later on, after some time, he reappears with a woman talking to him. That woman's testimony about him being the Messiah is key because it leads many Samaritans to believe in him.

Wait, there are also people from Galilee and other towns who heard about his story. They saw him visit Cana and turn water into wine, which is Jesus' trick to make people believe. Then he heals an official's son in Capernaum who was sick, which must have made the people believe that he was trustworthy.

So putting it all together, the woman's testimony about how he told everything she ever did seems like a st

Not entirely theologically correct, but also not too bad!

### Add chat interface

In [48]:
import gradio as gr

In [53]:
def invoke_model(message, history):
    response = qa_chain.invoke(message)
    result = response['result']
    return result

In [55]:
gr.ChatInterface(
    fn=invoke_model, 
    type="messages"
).launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
